# Execution Algorithms
Depending on the size of a trade being made, the trade itself can have an impact on the market. Indeed, when a firm buys a large amount of shares it can drive up the stock's price, just like selling a large number of shares can negatively impact the share price. To manage the problem of slippage, traders can use execution algorithms to split a large trade into multiple smaller trades which are executed over a certain time interval.

### TWAP vs VWAP
These algorithms are commonly used in trading to execute large orders in a controlled and efficient manner. Let's explore each of them in more detail:


#### TWAP (Time-Weighted Average Price)


TWAP is an execution algorithm that aims to achieve an average execution price that closely matches the average market price over a specified time period. The execution of the order is evenly spread out over this period, regardless of market conditions. This algorithm is particularly useful when the goal is to minimize the market impact of large orders by executing them gradually. The TWAP algorithm calculates the target order quantity to be executed in equal increments at regular time intervals. By dividing the order into smaller chunks and executing them at predetermined time intervals, TWAP aims to minimize price fluctuations caused by large order executions. It is important to note that TWAP execution does not take into account the volume traded at different time intervals.

In [1]:
from datetime import datetime, timedelta

def twap_execution(order, intervals: int, total_time: int):
    """
    Executes an order using the TWAP algorithm.
    
    Parameters:
        - order: Total order quantity to be executed (number of shares)
        - intervals: The number of intervals to be used
        - total_time: The total time for the order to be processed (in seconds)
    """
    #Implement method
    start_time = datetime.now()
    print('Order request received: {}'.format(start_time))
    print('----')
    interval_size = total_time / intervals
    order_size = order / intervals
    
    total_ordered = 0
    next_interval = start_time + timedelta(seconds=interval_size)
    while total_ordered < order:
        if datetime.now() >= next_interval:
            print('Order submitted for {n} shares at time {t}'.format(n=order_size, t=datetime.now()))
            total_ordered = total_ordered + order_size
            next_interval = next_interval + timedelta(seconds=interval_size)
            
    print('----')
    print('Order complete')
            
    
    
twap_execution(10000, 5, 10)

Order request received: 2023-05-31 17:28:42.041428
----
Order submitted for 2000.0 shares at time 2023-05-31 17:28:44.041433
Order submitted for 2000.0 shares at time 2023-05-31 17:28:46.041431
Order submitted for 2000.0 shares at time 2023-05-31 17:28:48.041431
Order submitted for 2000.0 shares at time 2023-05-31 17:28:50.041432
Order submitted for 2000.0 shares at time 2023-05-31 17:28:52.041431
----
Order complete


#### VWAP (Volume-Weighted Average Price)


VWAP is an execution algorithm that aims to achieve an average execution price that closely matches the volume-weighted average price of a security over a specified time period. The execution of the order is based on the volume traded at different price levels. VWAP execution is commonly used by institutional investors and traders who seek to execute large orders while minimizing the impact on the market. The VWAP algorithm calculates the target order quantity to be executed based on the proportion of the total traded volume at different price levels. This ensures that the execution is weighted according to the market activity. VWAP execution aims to achieve a price close to the average price at which the security has been traded throughout the day.



Both TWAP and VWAP algorithms provide systematic and controlled execution strategies for large orders. They help traders avoid excessive market impact by executing the orders gradually and in line with the prevailing market conditions. The choice between TWAP and VWAP depends on the specific requirements and objectives of the trader or investor.

It's worth noting that these algorithms are just a few examples of execution strategies, and there are other algorithms available, each with its own characteristics and benefits. Traders and investors often consider factors such as liquidity, market conditions, order size, and desired execution objectives when selecting the most appropriate algorithm for their trading needs.

In [2]:
import yfinance as yf

volumes = yf.download('F','2018-01-01',datetime.today(), progress=False)['Volume']

current = datetime.now()
volume_indices = []
for volume in volumes:
    current += timedelta(seconds=2)
    volume_indices.append(current)
volumes.index = volume_indices
volumes /= (8*60*30)
volumes.head()

2023-05-31 17:28:54.686591    1442.590278
2023-05-31 17:28:56.686591    2067.055556
2023-05-31 17:28:58.686591    2602.652778
2023-05-31 17:29:00.686591    3202.909722
2023-05-31 17:29:02.686591    2349.187500
Name: Volume, dtype: float64

In [12]:
def vwap_execution(order, total_time: int, volumes):
    """
    Executes an order using the TWAP algorithm.
    
    Parameters:
        - order: Total order quantity to be executed
        - volumes
    """
    #Implement method
    start_time = datetime.now()
    print('Order request received: {}'.format(start_time))
    completion_time = start_time + timedelta(seconds = total_time)
    volumes = volumes[volumes.index <= completion_time]
    volumes = volumes[volumes.index >= start_time]
    total_volume = sum(volumes)
    total_ordered = 0
    while total_ordered < order:
        if datetime.now() >= volumes.index[0]:
            order_size = order*(volumes[0]/total_volume)
            print('Order submitted for {n} shares at time {t}'.format(n = order_size, t=datetime.now()))
            total_ordered = total_ordered + order_size
            volumes = volumes[volumes.index > datetime.now()]
    print('-----')
    print('Order complete')
    
    
vwap_execution(10000, 10, volumes)

Order request received: 2023-05-31 17:35:12.694055
Order submitted for 2091.0354211922386 shares at time 2023-05-31 17:35:14.686637
Order submitted for 1729.3528311829223 shares at time 2023-05-31 17:35:16.686630
Order submitted for 1643.2000249022253 shares at time 2023-05-31 17:35:18.686628
Order submitted for 1941.3819401057008 shares at time 2023-05-31 17:35:20.686630
Order submitted for 2595.0297826169126 shares at time 2023-05-31 17:35:22.686631
-----
Order compelte


In [14]:
def pov_execution(order, volumes, participation=0.1):
    """
    """
    #Implement method
    start_time = datetime.now()
    print('Order request received: {}'.format(start_time))
    volumes = volumes[volumes.index >= start_time]
    total_ordered = 0
    while total_ordered < order:
        if datetime.now() >= volumes.index[0]:
            order_size = min(volumes[0]*participation, order-total_ordered)
            print('Order submitted for {n} shares at time {t}'.format(n = order_size, t=datetime.now()))
            total_ordered = total_ordered + order_size
            volumes = volumes[volumes.index > datetime.now()]
    print('-----')
    print('Order complete')
    
    
pov_execution(10000, volumes)

Order request received: 2023-05-31 17:43:18.579561
Order submitted for 260.35625 shares at time 2023-05-31 17:43:18.686626
Order submitted for 153.1326388888889 shares at time 2023-05-31 17:43:20.686628
Order submitted for 229.8090277777778 shares at time 2023-05-31 17:43:22.686634
Order submitted for 142.4326388888889 shares at time 2023-05-31 17:43:24.686631
Order submitted for 184.41875000000002 shares at time 2023-05-31 17:43:26.686625
Order submitted for 225.41319444444446 shares at time 2023-05-31 17:43:28.686629
Order submitted for 149.375 shares at time 2023-05-31 17:43:30.686634
Order submitted for 274.71875 shares at time 2023-05-31 17:43:32.686628
Order submitted for 473.17361111111114 shares at time 2023-05-31 17:43:34.686624
Order submitted for 283.52708333333334 shares at time 2023-05-31 17:43:36.686633
Order submitted for 193.24583333333334 shares at time 2023-05-31 17:43:38.686628
Order submitted for 203.7527777777778 shares at time 2023-05-31 17:43:40.686626
Order subm